In [ ]:
import transforms as transforms
import population as population
import optimize as optimize
import utility as utility
import pandas as pd
import numpy as np
import visualization as visualization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import evaluation as evaluation


data = pd.read_csv("../../data/ES15.csv")
x_raw = data.values

#x_raw, x_ind = train_test_split(x_raw_presplit, test_size=0.1, shuffle=False)

dirpath = utility.fetch_new_run_dirpath()



pop = population.generate_random_forest(100, 12)

np.seterr(all='ignore')
best_forest, best_scores, best_overtime = optimize.optimize_constants(
    pop, x_raw, sthresh_q=.1, run_dir=dirpath, max_iter=2, vizout=False
)

img = visualization.visualize_tree(best_forest[best_scores.index(min(best_scores))], run_dir=dirpath, vizout=True)
best_forest , best_scores = population.extract_n_best_trees(best_forest, best_scores, -1, run_dir=dirpath, vizout=True)


ynew = np.roll(x_raw[:, 3], shift=-1)
y_ = np.log(ynew / x_raw[:, 3])
#np.random.shuffle(y_)


x_ = transforms.forest2features(
    population=best_forest,
    x_raw=x_raw
)

#using time column (never utilized) as noise column
x_raw[:, 0] = 1
noise = np.random.rand(x_raw.shape[0], 1)

x_ = np.hstack([x_, noise])


X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3, shuffle=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = evaluation.standard_NN_construction(X_train, y_train, verbose=1)
evaluation.standard_LM_evaluation(X_train, X_test, y_train, y_test, model, dirpath, vizout=True, show=True)
#evaluation.standard_LM_evaluation(X_train, xind_, y_train, yind_, model, dirpath, vizout=True)

In [ ]:
import transforms as transforms
import population as population
import optimize as optimize
import utility as utility
import pandas as pd
import numpy as np
import visualization as visualization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import evaluation as evaluation


data = pd.read_csv("../../data/ES15.csv")
x_raw = data.values

#x_raw, x_ind = train_test_split(x_raw_presplit, test_size=0.1, shuffle=False)

dirpath = utility.fetch_new_run_dirpath()



pop = population.generate_random_forest(200, 12)

np.seterr(all='ignore')
best_forest, best_scores, best_overtime = optimize.optimize_constants(
    pop, x_raw, sthresh_q=.1, run_dir=dirpath, max_iter=-1, vizout=True
)

img = visualization.visualize_tree(best_forest[best_scores.index(min(best_scores))], run_dir=dirpath, vizout=True)
best_forest , best_scores = population.extract_n_best_trees(best_forest, best_scores, 10, run_dir=dirpath, vizout=True)


ynew = np.roll(x_raw[:, 3], shift=-1)
y_ = np.log(ynew / x_raw[:, 3])
#np.random.shuffle(y_)


x_ = transforms.forest2features(
    population=best_forest,
    x_raw=x_raw
)


#noise = np.random.rand(x_raw.shape[0], 1)
#x_ = np.hstack([x_, noise])


X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3, shuffle=True)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.optimizers.schedules import ExponentialDecay

def standard_NN_construction(X_train, y_train, epochs=250, verbose=0):
	

	reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
		monitor='val_loss',
		factor=0.65, 
		patience=3, 
		min_lr=1e-6
	)
	early_stopping = EarlyStopping(monitor='loss', patience=25, mode='min', restore_best_weights=True)

	opt  = tf.keras.optimizers.Adam()
	opt2 = tf.keras.optimizers.SGD(learning_rate=0.01)

	def build_model():
		model = tf.keras.Sequential([
			tf.keras.layers.Input(shape=(X_train.shape[1],)),
			tf.keras.layers.Dense(64, activation='relu'),
			tf.keras.layers.BatchNormalization(),
			tf.keras.layers.Dense(64, activation='relu'),  
			tf.keras.layers.Dropout(0.2),
			tf.keras.layers.Dense(64, activation='relu'),       
			tf.keras.layers.Dense(1, activation='linear')  # Output layer for regression
		])
		
		rmse='root_mean_squared_error'

		model.compile(optimizer=opt2, loss='mse', metrics=['R2Score'])
		return model

	with tf.device('/GPU:0'):
		model = build_model()
		history = model.fit(X_train, y_train, epochs=epochs, batch_size=512, \
						validation_split=0.2, verbose=verbose, shuffle=True, callbacks=[reduce_lr, early_stopping])
		
	return model, history

In [ ]:
import reproduction
best_forest = reproduction

In [ ]:
import transforms, evaluation
import pandas as pd
import numpy as np
data = pd.read_csv("../../data/ES15.csv")
x_raw = data.values
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

ynew = np.roll(x_raw[:, 3], shift=-1)
y_ = np.log(ynew / x_raw[:, 3])
#np.random.shuffle(y_)

import serialization

best_forest = serialization.load_forest(where='../../runs/run_5/best.4st')

x_ = transforms.forest2features(
    population=best_forest,
    x_raw=x_raw
)


#noise = np.random.rand(x_raw.shape[0], 1)
#x_ = np.hstack([x_, noise])


X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size=0.3, shuffle=True, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = evaluation.standard_LM_construction(X_train, y_train)
evaluation.standard_LM_evaluation(X_train, X_test, y_train, y_test, model, '', vizout=False, show=True)

In [ ]:
model, history = evaluation.standard_NN_construction(X_train, y_train, epochs=50, verbose=1)
evaluation.standard_NN_evaluation(X_train, X_test, y_train, y_test, model, history, run_dir='', vizout=False, show=True)

In [ ]:
%pip install numba

In [ ]:
import serialization
import matplotlib.pyplot as plt

path, scores = serialization.load_deeplist(where='../../runs/run_25/path_pscr.hstry')

plt.plot(range(len(scores)), scores)

In [ ]:
data = pd.read_csv("../../data/ES15.csv")
x_raw = data.values
best_forest = serialization.load_forest(where='../../runs/run_49/best.4st')
import optimize
x_ = transforms.forest2features(
	population=best_forest,
	x_raw=x_raw
)
_, __, best_scores = evaluation.evaluate_forest_newer(x_, close_prices=x_raw[:, 3], lag_range=(1, 3))
loss = optimize.loss_fc(best_scores)
print(loss)

for i in range(len(best_forest)):
	print(transforms.get_oplist(best_forest[i]))

In [ ]:
from sklearn.linear_model import LogisticRegression
# label train‐vs‐test rows
X_tv = np.vstack([X_train, X_test])
y_tv = np.array([0]*len(X_train) + [1]*len(X_test))
adv = LogisticRegression().fit(X_tv, y_tv)
print("AUC distinguishing train vs test:", adv.score(X_tv, y_tv))

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from transforms import forest2features

# 1) Define the transformer that wraps your feature generator
class Forest2Features(BaseEstimator, TransformerMixin):
    def __init__(self, population):
        self.population = population

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X is your raw x_raw array (n_samples, 4)
        return forest2features(self.population, X)

# 2) Prepare your raw data and target
#    Assume x_raw is an (N, 4) ndarray and best_forest is defined
#    Column 2 is the 'close' price
raw_prices = x_raw[:, 3]
# compute 1-step log-return target of length N-1
y_logret = np.log(raw_prices[1:] / raw_prices[:-1])
# align features by dropping the last raw row so X_raw_aligned.shape[0] == y_logret.shape[0]
X_raw_aligned = x_raw[:-1, :]

noise = np.random.rand(X_raw_aligned.shape[0], 1)

X_raw_aligned = np.hstack([X_raw_aligned, noise])

# 3) Build the pipeline
pipe = Pipeline([
    ("fe",     Forest2Features(population=best_forest)),
    ("scaler", StandardScaler()),
    ("lr",     LinearRegression())
])

# 4) Time-series cross-validation
tscv = TimeSeriesSplit(n_splits=3)
for fold, (train_idx, test_idx) in enumerate(tscv.split(X_raw_aligned), 1):
    X_tr, X_te = X_raw_aligned[train_idx], X_raw_aligned[test_idx]
    y_tr, y_te = y_logret     [train_idx], y_logret     [test_idx]

    #y_tr = np.random.permutation(y_tr)

    pipe.fit(X_tr, y_tr)
    
    perm = permutation_importance(pipe,
                              X_tr, y_tr,
                              n_repeats=5,
                              random_state=0,
                              scoring='r2')
    perm_importances = pd.Series(perm.importances_mean)
    print(perm_importances)
	#print("impt: ",perm_importances.max(),perm_importances[4])

    score = pipe.score(X_te, y_te)
    y_pred = pipe.predict(X_te)
    visualization.visualize_regression_eval(y_te, y_pred, show=False)
    print(f"Fold {fold} R²: {score:.4f}")


In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from transforms import forest2features

# 1) Define transformer wrapping your feature generator\class Forest2Features(BaseEstimator, TransformerMixin):
def __init__(self, population):
    self.population = population

def fit(self, X, y=None):
        return self

def transform(self, X):
        return forest2features(self.population, X)

# 2) Prepare raw data and target
#    Assume x_raw is an (N, 4) ndarray and best_forest is defined; column 2 is 'close'
raw_prices = x_raw[:, 3]
# compute 1-step log-return target of length N-1
y_logret = np.log(raw_prices[1:] / raw_prices[:-1])
# align features by dropping the last raw row so X_raw_aligned.shape[0] == y_logret.shape[0]
X_raw_aligned = x_raw[:-1, :]

# 3) Build pipeline
pipe = Pipeline([
    ("fe",     Forest2Features(population=best_forest)),
    ("scaler", StandardScaler()),
    ("lr",     LinearRegression())
])

# 4) Time-series cross-validation with post-feature-generation printing
tscv = TimeSeriesSplit(n_splits=5)
for fold, (train_idx, test_idx) in enumerate(tscv.split(X_raw_aligned), start=1):
    print(f"\n--- Fold {fold} ---")

    # raw train/test splits
    X_tr_raw, X_te_raw = X_raw_aligned[train_idx], X_raw_aligned[test_idx]
    y_tr,      y_te     = y_logret[train_idx],      y_logret[test_idx]

    # generate features using the transformer step
    fe = pipe.named_steps['fe']
    X_te_feats = fe.transform(X_te_raw)

    # inspect the generated features for the test split
    print("First 3 rows of generated test features:")
    print(X_te_feats[:3])
    print("Last 3 rows of generated test features:")
    print(X_te_feats[-3:])

    # fit the full pipeline (features -> scaler -> model)
    pipe.fit(X_tr_raw, y_tr)
    score = pipe.score(X_te_raw, y_te)
    print(f"Fold {fold} R²: {score:.4f}")


In [ ]:
dataext = pd.read_csv("../../data/ES15_ext.csv")
x_ext = dataext.values

xext_ = transforms.forest2features(
    population=best_forest,
    x_raw=x_ext
)

noise = np.random.rand(xext_.shape[0], 1)

xext_ = np.hstack([xext_, noise])


xext_ = scaler.transform(xext_)

ynewe = np.roll(x_ext[:, 3], shift=-1)
#np.random.shuffle(ynewe)
y_e = np.log(ynewe / x_ext[:, 3])

evaluation.standard_LM_evaluation(X_train, xext_, y_train, y_e, model, dirpath, vizout=True, show=True)


In [ ]:
scores = []

for i in range(6,x_raw.shape[0] -1):
    x_i = transforms.forest2features(
    	population=best_forest,
    	x_raw=x_raw[:i]
	)
    
    x_i[:, 0] = 1

    y_i = np.log( x_raw[1:i, 3] / x_raw[:i-1, 3] )
    
    x_i = np.hstack([x_i, np.random.rand(i, 1)])
    x_i = x_i[:-1]
    
    x_i = scaler.transform(x_i)
    
    print(model.score(x_i, y_i))

In [ ]:
x = pd.read_csv("../../data/ES15_ext.csv")
xr = x.values

x = transforms.forest2features(
    population=best_forest,
	x_raw=xr
)

x[:, 0] = 1
noise = np.random.rand(x.shape[0], 1)

x = np.hstack([x, noise])

x = scaler.transform(x)

ynewe = np.roll(xr[:, 3], shift=-1)
#np.random.shuffle(ynewe)
y = (ynewe / xr[:, 3]) - 1

pred = model.predict(x)

evaluation.standard_LM_evaluation(x, x, y, y, model, dirpath, vizout=True, show=True)


In [ ]:
mask = np.abs(model.predict(X_test)) > 0.0005
frac = mask.mean()
print(frac)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sign_accuracy_comparative(y_true, y_pred, show=True):
    """
    Plot two accuracy curves versus |y_pred| threshold, starting at 0.00025,
    and restrict x-axis to [0, 0.0025].

    - 'center': accuracy for predictions with |y_pred| <= threshold
    - 'extremes': accuracy for predictions with |y_pred| >= threshold

    Args:
        y_true: array-like of true values
        y_pred: array-like of predicted values
        show: whether to display the plot

    Returns:
        thresholds: filtered array of |y_pred| thresholds (>=0.00025)
        center_acc: accuracy for |y_pred| <= threshold
        extremes_acc: accuracy for |y_pred| >= threshold
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    abs_pred = np.abs(y_pred)
    true_sign = np.sign(y_true)
    pred_sign = np.sign(y_pred)

    # All unique thresholds, sorted ascending
    all_thresholds = np.unique(abs_pred)
    all_thresholds.sort()

    # Start at a more stable region: ignore thresholds below 0.00025
    thresholds = all_thresholds[all_thresholds >= 0.0001]

    # Compute accuracies at each filtered threshold
    center_acc = np.array([
        np.mean(true_sign[abs_pred <= t] == pred_sign[abs_pred <= t])
        for t in thresholds
    ])
    extremes_acc = np.array([
        np.mean(true_sign[abs_pred >= t] == pred_sign[abs_pred >= t])
        for t in thresholds
    ])

    if show:
        plt.figure()
        plt.plot(thresholds, center_acc, marker='o', linestyle='-',
                 label='|y_pred| ≤ threshold (center)')
        plt.plot(thresholds, extremes_acc, marker='^', linestyle='--',
                 label='|y_pred| ≥ threshold (extremes)')
        plt.xlabel("|y_pred| Threshold")
        plt.ylabel("Sign Prediction Accuracy")
        plt.title("Accuracy from Center vs. Extremes (from 0.00025)")
        plt.legend()
        plt.xlim(0.0001, 0.0025)
        plt.show()

    return thresholds, center_acc, extremes_acc


In [ ]:
y_pred_test = model.predict(X_test)
sign_accuracy_comparative(y_test, y_pred_test)


In [ ]:
_ = sign_accuracy_comparative(y, pred)

In [ ]:


for i in range(1,len(xr)):
    p = round(model.predict(x[:i])[-1], 5)
    if(abs(p) > 0.0005):
	    print(f'time: {xr[i, 0]}\tprice: {xr[i]}\tpred: {p}')